In [2]:
import numpy as np
import pandas as pd
from numpy.typing import NDArray
import matplotlib.pyplot as plt

# The market, and random movements explained

For next session!

# Parameter Estimation

We reviewed in our last session that the market can be modelled using a GBM, of the form:
$$ds = \mu S_t dt + \sigma S_t dX(t)$$
where $\quad dW_t$ is a draw from $N(0,1)$. We have found an analytical solution last session, but we aim to find numerical solutions and compare them to the analytical solution we found for our parameters. 

However, we did not discuss the methods from which we can obtain these parameters from real world data, and we will review two different methods for obtaining this. Hence, we wish to find the parameters $\mu$ and $\sigma$ that best describes the distribution. 

In [3]:
from pandas import Timestamp


ARRAY_F = NDArray[np.float64]
DEFAULT_DT = 1.0 /252

def create_weiner_array(size: int | tuple[int,...]) -> ARRAY_F:
    dX = np.random.normal(0, np.sqrt(DEFAULT_DT), size)
    X = dX.cumsum() 
    return X

def create_gbm(mu: float, sigma: float,  dt: float, s_0: float, W: ARRAY_F) -> ARRAY_F:
    S  = np.zeros_like(W)
    S[0] = s_0
    for i in range(1, len(W)):
        S[i] = mu * S[i-1] * dt + sigma * S[i-1] * W[i] 

    return S

We invite you to play around wiith these functions, and concretely produce 2 plots:
1. Families of paths with different parameter values for gbm
2. Average paths and path distributions at different points in time. 

What is evident from these distributions?

# Maximum Likelihood Estimation (MLE)

We wish to find the parameters $\mu$ and $\sigma$ that maximise the total probability that the observed data was drawn from the distribution generated with parameters $\mu$ and $\sigma$. Recall that GBM has an analytical solution:
$$S_t = S_0 \text{exp}\left((\mu - \frac{1}{2}\sigma^2)t + \sigma W_t\right)$$
Let $X = \ln{S}$, hence we find that:

$$ dX_t = X_{t+dt}-X_t = \ln  {S_{t+dt} - \ln S_t} = \ln\left(\frac{S_{t+dt}}{S_t}\right)$$
which simplifies to:

$$dX_t = (\mu - \frac{1}{2}\sigma^2)dt + \sigma dW_t$$
we can conclude that:
$$dX ∼ \mathcal{N}  ((\mu - \frac{1}{2}\sigma^2) \Delta t_i , \sigma^2 \Delta t_i)$$
From the normal distribution function, letting $m = \mu - \frac{1}{2}\sigma^2$, we obtain that the probability density of $(Y_i)_{i=0}^n$ is:

$$
L(\mu,\sigma)
=\prod_{i=1}^n
\frac{1}{\sqrt{2\pi\,\sigma^2\Delta t_i}}
\exp\!\left(
-\frac{\big(Y_i-m\Delta t_i\big)^2}{2\,\sigma^2\Delta t_i}
\right)
$$
and hence the log-likelihood is described by:
$$
\ell(\mu,\sigma):=\ln L(\mu,\sigma)
= -\tfrac12\sum_{i=1}^n
\left[
\ln\!\big(2\pi\,\sigma^2\Delta t_i\big)
+\frac{\big(Y_i-m\Delta t_i\big)^2}{\sigma^2\Delta t_i}
\right].
$$
Thus, we aim to maximise the log likelihood of the parameters. We can find the analytical solution by setting partial derivatives equal to zero, below are the equations for the solutions:
$$
\sigma^2 = \frac{1}{n}\sum_{i=1}^n \frac{(Y_i - \hat{m}\Delta t_i)^2}{\Delta t_i}
\\
\mu = \hat{m} + \frac{1}{2}\sigma^2 \\
\text{where:} \\ \\ \\
\hat{m} = \frac{\sum_{i=1}^n Y_i}{\sum_{i=0}^n \Delta t_i}


## Function optimiser:
$$
\ell(\mu,\sigma)
= -\tfrac12\sum_{i=1}^n
\left[
\ln(\sigma^2)
+\frac{\big(Y_i-m\Delta t_i\big)^2}{\sigma^2\Delta t_i}
\right]
+ O(1)
$$

In [10]:
def log_likelihood(mu: float, sigma: float, equity_path: ARRAY_F):
        m = mu - (1/2) * sigma * sigma
        ll_sum = 0
        for i in range(len(equity_path)):
            ll_sum += np.log(sigma*sigma) + ((equity_path[i] - m*DEFAULT_DT)**2 / (sigma*sigma*DEFAULT_DT))

        ll_sum_adjusted = (-1/2) * ll_sum
        return ll_sum_adjusted
            

def optimise_params_mle_analytical(equity_path: ARRAY_F) -> tuple[float, float]:
    prices = np.asarray(equity_path, dtype=np.float64)
    if prices.ndim != 1:
        raise ValueError("equity_path must be a 1D array of prices")
    if prices.size < 2:
        raise ValueError("equity_path must contain at least two prices")
    if not np.all(prices > 0.0):
        raise ValueError("all prices must be strictly positive")

    log_prices = np.log(prices)
    log_returns = np.diff(log_prices)

    n_observations = log_returns.size

    mean_log_return = float(np.mean(log_returns))
    centered_log_returns = log_returns - mean_log_return

    # \hat{\sigma}^2 = (1/(n \Delta t)) \sum (Y_i - \bar Y)^2
    sigma_squared_hat = float((centered_log_returns @ centered_log_returns) / (n_observations * DEFAULT_DT))
    if sigma_squared_hat <= 0.0:
        raise ValueError("estimated variance is non-positive; check input data")

    sigma_hat = float(np.sqrt(sigma_squared_hat))

    # \hat{\mu} = \bar Y / \Delta t + (1/2) \hat{\sigma}^2
    mu_hat = float(mean_log_return / DEFAULT_DT + 0.5 * sigma_squared_hat)

    return mu_hat, sigma_hat

def optimise_params_mle_numerical(
    equity_path: ARRAY_F,
    mu_bounds: tuple[float, float] = (-2.0, 2.0),
    sigma_bounds: tuple[float, float] = (1e-6, 2.0),
    mu_grid_size: int = 301,
    sigma_grid_size: int = 301,
) -> tuple[float, float]:
    prices = np.asarray(equity_path, dtype=np.float64)
    if prices.ndim != 1:
        raise ValueError("equity_path must be a 1D array of prices")
    if prices.size < 2:
        raise ValueError("equity_path must contain at least two prices")
    if not np.all(prices > 0.0):
        raise ValueError("all prices must be strictly positive")

    mu_grid = np.linspace(mu_bounds[0], mu_bounds[1], mu_grid_size, dtype=np.float64)
    sigma_grid = np.linspace(sigma_bounds[0], sigma_bounds[1], sigma_grid_size, dtype=np.float64)

    best_mu = 0.0
    best_sigma = 0.0
    best_ll = -np.inf

    for sigma_candidate in sigma_grid:
        for mu_candidate in mu_grid:
            ll_value = log_likelihood(mu_candidate, sigma_candidate, prices)
            if ll_value > best_ll:
                best_ll = ll_value
                best_mu = mu_candidate
                best_sigma = sigma_candidate

    return float(best_mu), float(best_sigma)

# Project

Implement the functions below, and for an added difficulty; derive and recreate the functions above. We will expand on this later!

In [ ]:


def model_equity(ticker: str, start_date: Timestamp, end_date: Timestamp) -> ARRAY_F:
    ...

def simulate_equity_matrix(tickers: list[str]):
    ... 

def retrieve_equity_matrix(tickers: list[str]):
    ...